The following notebook loads and runs the pretrained model on pre prepared data. The code is saved in a Google Drive folder and such must be accessed. The pretrained model, and requirements files were all edited to fit the needs of Venzi. Some files were renamed. 


Locate the Google drive where the pretrained model and data will be downloaded

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [12]:
%cd /content/gdrive/My Drive/INFO 3604 FINAL/ReviewSummarizer/OpinionMining

/content/gdrive/My Drive/INFO 3604 FINAL/ReviewSummarizer/OpinionMining


In [13]:
%cd /content/gdrive/My Drive/INFO 3604 FINAL/ReviewSummarizer/OpinionMining/opiniondigest 

/content/gdrive/My Drive/INFO 3604 FINAL/ReviewSummarizer/OpinionMining/opiniondigest


Download pretrained model. This download file gets the zipped model from megagonlabs

In [ ]:
!bash download.sh

--2021-04-15 21:41:39--  https://opiniondigest.s3-us-west-2.amazonaws.com/data/yelp-default-data.zip
Resolving opiniondigest.s3-us-west-2.amazonaws.com (opiniondigest.s3-us-west-2.amazonaws.com)... 52.218.244.217
Connecting to opiniondigest.s3-us-west-2.amazonaws.com (opiniondigest.s3-us-west-2.amazonaws.com)|52.218.244.217|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 510580108 (487M) [application/zip]
Saving to: ‘yelp-default-data.zip’

yelp-default-data.z 100%[===================>] 486.93M  74.7MB/s    in 6.6s    

2021-04-15 21:41:46 (73.5 MB/s) - ‘yelp-default-data.zip’ saved [510580108/510580108]

Archive:  yelp-default-data.zip
replace data/yelp-default/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/yelp-default/test_gold_8_15_all_all_300_8.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/yelp-default/summaries_0-200_cleaned_fixed_business_ids.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/yelp-default/test.csv? [y

In [31]:
%cd /content/gdrive/My Drive/INFO 3604 FINAL/ReviewSummarizer/OpinionMining/opiniondigest 

/content/gdrive/My Drive/INFO 3604 FINAL/ReviewSummarizer/OpinionMining/opiniondigest


Install requirements file which holds the libraries and packages needed to run the model. 
The requirements.txt initially had
torch==1.2.0
torchtext==0.4.0

It was changed to
torch==1.4.0 
torchvision==0.5.0
torchtext==0.4.0

To deal with errors that came up with torch and memory issues while running the code in a GPU

In [15]:
!pip install -r requirements.txt

     |████████████████████████████████| 81kB 5.0MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
  Created wheel for sumeval: filename=sumeval-0.2.2-cp37-none-any.whl size=54536 sha256=8c752306e97d3ffceef3d19645fb42b478aaf52027c65902255a783352c59b7e
  Stored in directory: /root/.cache/pip/wheels/7b/6f/57/19ceecab21445c88f3c565735fa1887b4cd18d340c972eb445
Successfully built sumeval


Generate summaries of reviews

In [16]:
%run src/generate.py \
  config/prepare_yelp-default.json \
  config/train_default.json \
  ./data/yelp-default/test_gold_8_15_all_all_300_8.csv \
  config/generate_beam-5-06-3-60.json

Display some of the summaries generated. The file displays the eid of the store/hotel, the input text into the model which is the extracted opinions and aspects and the generated summary

In [17]:
!cat output/yelp-default_op2text_default_8-15-all-all-300-8_beam-5-06-3-60_agg.csv

eid,input_text,pred
3cbsPfoUUrysf-M8FI_0IA,fantastic Saturday brunch [SEP] immediate coffee [SEP] friendly prompt service [SEP] delicious chorizo [SEP] homemade chickpea gravy [SEP] warm berry white donuts [SEP] cozy atmosphere [SEP] good food [SEP] very savory [SEP] Nice neighborhood [SEP] Lavender Lemon `` donuts [SEP] nice Ambiance [SEP] much smaller restaurant [SEP] super pleasant Wait staff [SEP] very unique tastes,this is a nice neighborhood restaurant with a nice ambiance and the staff is super friendly and the service is prompt. the food is very good and the coffee is fantastic. i had the french toast and it was delicious. the homemade french toast is a unique combination of sweet and savory and savory french toast. the coffee was
5iHctUjkQTGwEvOaBkwMRQ,favorite restaurant for lunch [SEP] soo good coffee [SEP] great teas [SEP] amazing quality selection of [SEP] delicious pesto chicken pannini [SEP] super nice staff [SEP] really sweet girls [SEP] toasted marshmallow iced latte [

Evalute summaries generated with ROUGE and BLEU metrics

In [32]:
%run src/evaluate.py \
  config/prepare_yelp-default.json \
  config/train_default.json \
  config/aggregate_8_15_all_all.json \
  config/generate_beam-5-06-3-60.json

Output of the evaluation of the summaries generated

In [33]:
!cat output/yelp-default_op2text_default_8_15_all_all_beam-5-06-3-60_agg.eval

,0
rouge_1,0.2671144131957425
rouge_2,0.04654657411928564
rouge_l,0.1761376959284657
